In [ ]:
from pyspark.sql import SparkSession
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
import geopandas as gpd

spark = SparkSession. \
    builder. \
    appName('Milestone 2 Data Prep'). \
    master("spark://parallels-Parallels-Virtual-Platform:7077"). \
    config("spark.serializer", KryoSerializer.getName). \
    config("spark.kryo.registrator", SedonaKryoRegistrator.getName). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-python-adapter-3.0_2.12:1.0.1-incubating,'
           'org.datasyslab:geotools-wrapper:geotools-24.1'). \
    getOrCreate()

SedonaRegistrator.registerAll(spark)

21/10/07 16:48:57 WARN Utils: Your hostname, parallels-Parallels-Virtual-Platform resolves to a loopback address: 127.0.1.1; using 10.211.55.3 instead (on interface enp0s5)
21/10/07 16:48:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/parallels/.ivy2/cache
The jars for the packages stored in: /home/parallels/.ivy2/jars
:: loading settings :: url = jar:file:/home/parallels/.local/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.sedona#sedona-python-adapter-3.0_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e6d9abbc-5dfc-4d3e-8182-533c36d14123;1.0
	confs: [default]
	found org.apache.sedona#sedona-python-adapter-3.0_2.12;1.0.1-incubating in central
	found org.locationtech.jts#jts-core;1.18.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found com.fasterxml.j

21/10/07 16:55:25 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
21/10/07 16:55:40 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
21/10/07 16:55:55 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [ ]:
#Airport DF
airports_dat = spark.read.format("csv").option("header", "true").option("delimiter", ",").load("airports.dat").toDF("Airport_ID", "Name", "City", "Country", "IATA", "ICAO", "Latitude", "Longitude", "Altitude", "Timezone", "DST", "Tz Database time zone", "Type", "Source")
#airports_dat.show(1)
airportsDF = gpd.GeoDataFrame(airports_dat.toPandas())
airportsDF

In [ ]:
state_boundary_tsv = spark.read.option("delimiter", "\t").option("header", "false").csv("boundary-each-state.tsv").toDF("s_name","s_bound")
#state_boundary_tsv.show()
state_boundaryDF = gpd.GeoDataFrame(state_boundary_tsv.toPandas())

states = state_boundary_tsv.selectExpr("s_name", "ST_GeomFromWKT(s_bound) as s_bound")
states.printSchema()
state_boundaryDF

In [ ]:
airlines_dat = spark.read.format("csv").option("header", "true").option("delimiter", ",").load("airlines.dat")
#airlines_dat.show()
airlinesDF = gpd.GeoDataFrame(airlines_dat.toPandas())
airlinesDF.columns = ["AirlineID", "Name", "Alias", "IATA", "ICAO", "Callsign", "Country", "Active"]
airlinesDF

In [ ]:
routes_dat = spark.read.format("csv").option("header", "true").option("delimiter", ",").load("routes.dat").toDF("Airline", "Airline_ID", "Source_Airport", "Source_Airport_ID", "Destination_Airport", "Destination_Airport_ID", "Codeshare", "Stops", "Equipment")
#routes_dat.show()
#routes_dat.printSchema()

routesDF = gpd.GeoDataFrame(routes_dat.toPandas())
routesDF